In [29]:
import shapely
from shapely.ops import nearest_points
import numpy as np
import matplotlib.pyplot as plt
from environment import *
from timeit import timeit

In [4]:
std_env = "simple"
with open(f"./levels/{std_env}.json", "r") as file:
        json_str = file.read()
        env = Environment.from_json(json_str, False)

In [19]:
points = shapely.MultiPoint([(0.0, 1.0),(3.0, 7.0),(5.0, 10.0),])
points.geoms

In [41]:
def shape_intersects():
    poly = shapely.unary_union([shapely.LinearRing(obs.translate_corners()) for obs in env.obstacles])
    robo_pos = env.get_robo_pos()
    robo_point = shapely.Point(robo_pos)
    robo_angle = env.get_robo_angle()
    sens_dist = np.array([500,0])
    angles = np.linspace(robo_angle, np.pi * 2 + robo_angle, 360)
    directions = [rotate(sens_dist, angle) for angle in angles]
    lines = [shapely.LineString([robo_pos, robo_pos + direc]) for direc in directions]
    intersects = [line.intersection(poly) for line in lines]
    closest_poss = [robo_pos + directions[i] if inter.is_empty else np.array(nearest_points(inter, robo_point)[0].coords) for i, inter in enumerate(intersects)]
    distances = [np.linalg.norm(s - robo_pos) for s in closest_poss]
    return distances

def shape_intersects2():
    poly = shapely.unary_union([shapely.LinearRing(obs.translate_corners()) for obs in env.obstacles])
    robo_pos = env.get_robo_pos()
    robo_point = shapely.Point(robo_pos)
    robo_angle = env.get_robo_angle()
    sens_dist = np.array([500,0])
    angles = np.linspace(robo_angle, np.pi * 2 + robo_angle, 360)
    out = []
    for angle in angles:
        direction = rotate(sens_dist, angle)
        line = shapely.LineString([robo_pos, robo_pos + direction])
        intersection = line.intersection(poly)
        if intersection.is_empty:
            closest_pos = robo_pos + direction 
        else: 
            closest_pos = np.array(nearest_points(intersection, robo_point)[0].coords)
        out.append(np.linalg.norm(closest_pos - robo_pos))
    return out

print(shape_intersects())


[163.09818750207492, 161.22760992659485, 159.4477397139507, 157.75453213334637, 156.14423353214872, 154.61335830810856, 153.15866815557223, 151.77715333540516, 150.4660157498344, 149.2226536305492, 148.04464767184493, 146.929748460908, 145.87586507497275, 144.88105473043757, 143.94351338242353, 143.06156718497758, 142.2336647323982, 141.45837001119372, 140.73435600014275, 140.06039886295815, 139.43537268428562, 138.85824470530218, 138.32807102011034, 137.84399269853373, 137.40523230487335, 137.011090785745, 136.66094470334028, 136.35424379337746, 136.0905088296839, 135.86932977980442, 135.69036423830008, 135.55333612651586, 135.45803464957802, 135.40431350326008, 135.39209032514705, 135.42134638626246, 135.4921265210047, 135.60453929490566, 135.75875741137995, 135.9550183603067, 136.1936253129832, 136.47494826974892, 136.79942546839973, 137.1675650634322, 137.5799470881891, 138.03722571415443, 138.5401318239861, 139.08947591741364, 139.68615137190486, 140.33113808304049, 141.0255065128

In [40]:
n = 500
print(f"environ: {timeit(env.get_distance_scans, number=n) / n}s")
print(f"shapel1: {timeit(shape_intersects, number=n) / n}s")
print(f"shapel2: {timeit(shape_intersects, number=n) / n}s")

environ: 0.05368106439999246s
shapel1: 0.031402204199999686s
shapel2: 0.03146913660000428s
